In [1]:
from pathlib import Path
import numpy as np
from keras.preprocessing import image
from keras.applications import vgg16
from keras.layers import Dense, Dropout, Flatten

Using TensorFlow backend.


In [2]:
# Path to folders with training data
dog_path = Path("training_data") / "dogs"
not_dog_path = Path("training_data") / "not_dogs"

images = []
labels = []

In [3]:
# Load all the not-dog images
for img in not_dog_path.glob("*.png"):
    # Load the image from disk
    img = image.load_img(img)

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)

    # For each 'not dog' image, the expected value should be 0
    labels.append(0)

In [4]:
# Load all the dog images
for img in dog_path.glob("*.png"):
    # Load the image from disk
    img = image.load_img(img)

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)

    # For each 'dog' image, the expected value should be 1
    labels.append(1)

In [5]:
from sklearn.utils import shuffle
images, labels = shuffle(images, labels)


In [6]:
X = np.array(images)

# Also convert the labels to a numpy array
y = np.array(labels)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



In [7]:
X_train.shape

(38, 64, 64, 3)

In [8]:
# Normalize image data to 0-to-1 range
X_train = vgg16.preprocess_input(X_train)
X_test = vgg16.preprocess_input(X_test)

In [9]:
# Load a pre-trained neural network to use as a feature extractor
# set include_top = False to remove the top layer to be replaced by our own classifier
base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [12]:
# Define the layers in the new classification prediction 
from keras import Model
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)        # new FC layer, random init
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)  # new softmax layer

# Define trainable model which links input from the  base model to the new classification prediction layers
model = Model(inputs=base_model.input, outputs=predictions)

# print model structure diagram
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [13]:
# Transfer Learning
print ('\nPerforming Transfer Learning')
  #   Freeze all layers in the Inception V3 base model 
for layer in base_model.layers:
  layer.trainable = False
#   Define model compile for basic Transfer Learning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Performing Transfer Learning


In [14]:
# Train the model
model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=8,
    validation_data=(X_test,y_test)
)

Instructions for updating:
Use tf.cast instead.
Train on 38 samples, validate on 20 samples
Epoch 1/10
38/38 [==============================] - 4s 110ms/step - loss: 8.4831 - acc: 0.3158 - val_loss: 4.4981 - val_acc: 0.5000
Epoch 2/10
38/38 [==============================] - 3s 84ms/step - loss: 4.2063 - acc: 0.6842 - val_loss: 3.1936 - val_acc: 0.7000
Epoch 3/10
38/38 [==============================] - 4s 98ms/step - loss: 2.5353 - acc: 0.8158 - val_loss: 5.5452 - val_acc: 0.6500
Epoch 4/10
38/38 [==============================] - 3s 92ms/step - loss: 1.6839 - acc: 0.8947 - val_loss: 5.5800 - val_acc: 0.6500
Epoch 5/10
38/38 [==============================] - 3s 91ms/step - loss: 1.7846 - acc: 0.8684 - val_loss: 5.5249 - val_acc: 0.6500
Epoch 6/10
38/38 [==============================] - 4s 94ms/step - loss: 0.4195 - acc: 0.9737 - val_loss: 4.4331 - val_acc: 0.7000
Epoch 7/10
38/38 [==============================] - 3s 89ms/step - loss: 0.6972 - acc: 0.9474 - val_loss: 4.0989 - val_ac

In [15]:
model.save("transferlearningmodel.h5")